# Importazione librerie

In [1]:
import pandas as pd

# Importazione e conversione datasets

Data la grande dimensione di alcuni dei dataset utilizzati, questi verranno convertiti da `.csv` a `.parquet` (Apache Parquet Format)

In [10]:
#df = pd.read_csv('./Datasets/Dati_sensori_meteo_2021_parte_2.csv', low_memory=False)
#df.to_parquet('./Datasets/Dati_sensori_meteo_2021_parte_2.parquet')

In [11]:
meteo_21 = pd.read_parquet('./Datasets/Dati_sensori_meteo_2021_parte_2.parquet')
meteo_21.head()

,IdSensore,Data,Valore,idOperatore,Stato
0,19088,01/12/2021 01:40:00 AM,2.7,3,VA
1,9101,01/12/2021 01:10:00 AM,0.0,4,VA
2,9123,01/12/2021 01:10:00 AM,0.0,4,VA
3,14047,01/12/2021 02:30:00 AM,250.0,3,VA
4,2459,01/12/2021 02:50:00 AM,2.0,1,VA


In [12]:
aria_21 = pd.read_csv('./Datasets/Dati_sensori_aria_2021.csv')
aria_21.head()

,IdSensore,Data,Valore,Stato,idOperatore
0,5832,01/01/2021 12:00:00 AM,-9999.0,NaN,1
1,5832,01/01/2021 04:00:00 AM,-9999.0,NaN,1
2,6328,01/01/2021 03:00:00 AM,-9999.0,NaN,1
3,5504,01/01/2021 04:00:00 AM,-9999.0,NaN,1
4,5504,01/01/2021 03:00:00 AM,-9999.0,NaN,1


In [53]:
stazioni_meteo = pd.read_csv('./Datasets/Stazioni_Meteorologiche.csv', dtype = {'Tipologia': 'category'})
stazioni_meteo.head()
#stazioni_meteo['Tipologia'].value_counts()

Precipitazione         281
Temperatura            258
Umidità Relativa       199
Direzione Vento        144
Velocità Vento         144
Radiazione Globale     110
Livello Idrometrico     85
Altezza Neve            41
Name: Tipologia, dtype: int64

In [54]:
stazioni_aria = pd.read_csv('./Datasets/Stazioni_qualita__dell_aria.csv', dtype = {'NomeTipoSensore': 'category'})
stazioni_aria.head()
stazioni_aria['NomeTipoSensore'].value_counts()

Biossido di Azoto             166
Ossidi di Azoto               166
Biossido di Zolfo             125
Monossido di Carbonio         101
Ozono                          91
PM10 (SM2005)                  90
Particolato Totale Sospeso     55
Particelle sospese PM2.5       40
Benzene                        32
Nikel                          15
Piombo                         15
Arsenico                       15
Cadmio                         15
Benzo(a)pirene                 15
Ammoniaca                      14
PM10                            4
BlackCarbon                     4
Monossido di Azoto              3
Name: NomeTipoSensore, dtype: int64

In [58]:
zonizzazione = pd.read_csv('./Datasets/classificazione_comuni_ABCD.csv', dtype = {'Zona': 'category'})
zonizzazione.head()
#sum(zonizzazione['Residenti 2008'])
#zonizzazione['Zona'].value_counts()

,Provincia,Codice Istat,Comune,Zona,Residenti 2008,Superficie (ha)
0,BG,16009,AMBIVERE,A,2341,327
1,BG,16013,ARZAGO D'ADDA,A,2836,944
2,BG,16018,BAGNATICA,A,4119,639
3,BG,16020,BARIANO,A,4396,714
4,BG,16021,BARZANO',A,5178,356


### Riduzione dimensioni

Meteo: 343 stazioni con 1262 sensori

Aria: 174 stazioni con 966 sensori

Zonizzazione: 9 718 786 abitanti in 1521 comuni divisi in 7 o 8 zone

- in `stazioni_aria` e `stazioni_meteo` tenere solo righe con `Stato == VA`
- da `stazioni_aria` e `stazioni_meteo` trovare solo gli `IdSensore` di tipologia utile
- tenere solo stazioni meteo dove ci sono stazioni aria (joint su `lat` e `lng`?)
- inner joint tra `aria_xx` e `meteo_xx` con stazioni filtrate
- left joint (non case sensitive) con dataset `zonizzazione` rispetto a `Comune` tenendo solo la colonna `Zona`
- salvare i nuovi dataset in formato parquet